# Tweets Analysis

In [134]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd
import re
import requests
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from langdetect import detect_langs
from fuzzywuzzy import fuzz

## Tweets scraping
### 1. Extract and parse original data

In [2]:
df = pd.read_excel('ProjectList.xls')

# get rid of spaces inside ICO names
name_strip = []
for name in df['name'].tolist():
    name_strip.append(name.replace(' ', ''))
df.name = name_strip

#df.columns
df = df[['name', 'PreICOtimestart', 'PreICOtimeend', 'preICOstart', 'preICOend', 'ICOTimestart', 'ICOTimeend', 'ICOstart', 'ICOend']]
df = df.astype(object).where(df.notnull(), None) # change data type to None

# parse date into the format when scraping on twitter
## ordinal date format
def ordinal_parser(col):
    date = []
    for item in col:
        if item != None:
            date.append(datetime.strptime(item.replace(item.split(' ')[0], re.findall('.\d*', item)[0]), '%d %b %Y').strftime('%Y-%m-%d'))
        else:
            date.append(None)
    return date
### these four attributes are in ordinal format due to the source website
df.preICOstart = ordinal_parser(df.preICOstart)
df.preICOend = ordinal_parser(df.preICOend)
df.ICOstart = ordinal_parser(df.ICOstart)
df.ICOend = ordinal_parser(df.ICOend)

## date time format
def datetime_parser(col):
    date = []
    for item in col:
        if item != None:
            date.append(datetime.date(item).strftime('%Y-%m-%d'))
        else:
            date.append(None)
    return date
### these four attributes are in datetime format due to another source website
df.PreICOtimestart = datetime_parser(df.PreICOtimestart)
df.PreICOtimeend = datetime_parser(df.PreICOtimeend)
df.ICOTimestart = datetime_parser(df.ICOTimestart)
df.ICOTimeend = datetime_parser(df.ICOTimeend)

df.shape
df.head(10)

(1876, 9)

,name,PreICOtimestart,PreICOtimeend,preICOstart,preICOend,ICOTimestart,ICOTimeend,ICOstart,ICOend
0,8CircuitStudios,None,None,2018-09-30,2018-10-01,None,None,2018-10-01,2018-10-31
1,88Coin,None,None,None,None,None,None,None,None
2,Aditus,None,None,None,None,None,None,None,None
3,aelf,None,None,None,None,None,None,None,None
4,AERGO,None,None,None,None,None,None,None,None
5,Agora,None,None,None,None,None,None,None,None
6,Agora,None,None,None,None,None,None,None,None
7,Ahoolee,None,None,None,None,None,None,2017-08-28,2017-09-11
8,Aion,None,None,None,None,None,None,None,None
9,Aira,None,None,None,None,None,None,None,None


### 2. Data preparation
#### a) pre-sale stage

In [3]:
# navigate to ICOs which have date info in their pre-sale stage
df_pre = df.iloc[:,0:5]
#df_pre.head(10)

# check if there is any observation that contains infomation from both sources
## start
for ind1 in df_pre[df_pre.PreICOtimestart.isna() == False].index:
    for ind2 in df_pre[df_pre.preICOstart.isna() == False].index:
        if ind1 == ind2:
            print('match found in starts')
            break
## end
for ind1 in df_pre[df_pre.PreICOtimeend.isna() == False].index:
    for ind2 in df_pre[df_pre.preICOend.isna() == False].index:
        if ind1 == ind2:
            print('match found in ends')
            break

# no overlaps, then merge starts and ends together
d1 = df_pre[(df_pre.PreICOtimestart.isna() == False) & (df_pre.PreICOtimeend.isna() == False)].iloc[:, 0:3]
d2 = df_pre[(df_pre.preICOstart.isna() == False) & (df_pre.preICOend.isna() == False)].iloc[:, [0, 3, 4]]
#d1.head()
#d2.head()
name = pd.concat([d1.name, d2.name], ignore_index = True)
start = pd.concat([d1.PreICOtimestart, d2.preICOstart], ignore_index = True)
end = pd.concat([d1.PreICOtimeend, d2.preICOend], ignore_index = True)

df_pre = pd.concat([name, start, end], axis = 1)
df_pre.columns = ['name', 'pre_start', 'pre_end']
df_pre = df_pre.sort_values(by = ['pre_start', 'pre_end'], ascending = [True, True]).reset_index(drop = True)

df_pre.shape
df_pre.head()

(807, 3)

,name,pre_start,pre_end
0,BitparkCoin,2016-07-25,2016-08-15
1,XinFin,2017-06-01,2017-07-31
2,Cofound.it,2017-06-05,2017-06-07
3,Populous,2017-06-08,2017-07-08
4,Kapu,2017-06-15,2017-08-31


#### b) sale stage

In [4]:
# navigate to ICOs which have date info in their sale stage
df_sale = df.iloc[:,[0, 5, 6, 7, 8]]
#df_sale.head(10)

# check if there is any observation that contains infomation from both sources
## start
for ind1 in df_sale[df_sale.ICOTimestart.isna() == False].index:
    for ind2 in df_sale[df_sale.ICOstart.isna() == False].index:
        if ind1 == ind2:
            print('match found in starts')
            break
## end
for ind1 in df_sale[df_sale.ICOTimeend.isna() == False].index:
    for ind2 in df_sale[df_sale.ICOend.isna() == False].index:
        if ind1 == ind2:
            print('match found in ends')
            break

# no overlaps, then merge starts and ends together
d1 = df_sale[(df_sale.ICOTimestart.isna() == False) & (df_sale.ICOTimeend.isna() == False)].iloc[:, 0:3]
d2 = df_sale[(df_sale.ICOstart.isna() == False) & (df_sale.ICOend.isna() == False)].iloc[:, [0, 3, 4]]
#d1.head()
#d2.head()
name = pd.concat([d1.name, d2.name], ignore_index = True)
start = pd.concat([d1.ICOTimestart, d2.ICOstart], ignore_index = True)
end = pd.concat([d1.ICOTimeend, d2.ICOend], ignore_index = True)

df_sale = pd.concat([name, start, end], axis = 1)
df_sale.columns = ['name', 'sale_start', 'sale_end']
df_sale = df_sale.sort_values(by = ['sale_start', 'sale_end'], ascending = [True, True]).reset_index(drop = True)

df_sale.shape
df_sale.head()

(1755, 3)

,name,sale_start,sale_end
0,BitCrystals,2015-08-04,2015-09-03
1,Augur,2015-08-17,2015-10-01
2,DigixDAO,2016-03-29,2016-03-30
3,Waves,2016-04-12,2016-05-31
4,Newbium,2016-05-01,2016-05-31


### 3. Scrape tweets in both stages

In [5]:
# function to parse tweets html using BeautifulSoup
def getTweets(tweets_html):
    result=[]
    
    soup=BeautifulSoup(tweets_html, 'html.parser')

    tweets=soup.select('div.js-stream-tweet')

    for t in tweets:
        ICO_name, username, text, timestamp, reply, retweet, like, lang = '', '', '', '', '', '', '', ''
        
        ICO_name = ico_name # come from the global variable in the next function
        
        select_user = t.select('span.username.u-dir b')
        if select_user != []:
            username = select_user[0].get_text()

        select_text = t.select('p.js-tweet-text')
        if select_text != []:
            text = select_text[0].get_text()
    
        select_time = t.select('small.time span.js-short-timestamp')
        if select_time != []:
            timestamp = int(select_time[0]['data-time'])
            timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        
        select_stream_item_footer = t.select('span.ProfileTweet-actionCountForPresentation')
        if select_stream_item_footer != []:
            reply = select_stream_item_footer[0].get_text()
            retweet = select_stream_item_footer[1].get_text()
            like = select_stream_item_footer[3].get_text()
        
        select_language = t.select('p.tweet-text')
        if select_language != []:
            lang = select_language[0]['lang']
        
        result.append({'ICO_name':ICO_name, "user":username, "text":text, "date":timestamp,\
                       'reply':reply, 'retweet':retweet, 'like':like, 'language':lang})
        
    return result

In [6]:
# function to scrape and save tweets from the results of function getTweets
def tweets_scraper(data):    
    # User agent must be defined in http request header
    # a user agent is software that is acting on behalf of a user. Usually it tells the browser used.
    # some websites reject requests without a user agent
    headers = { 'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.'
                                  '86 Safari/537.36'}

    # the search returns tweets in the decending order of time, set the max_position to the min_position of last search
    min_position = '1'
    min_position_updated = ''
    tweets_scraped = []

    for ind, row in data.iterrows():
        while min_position != min_position_updated:
            min_position = min_position_updated
            global ico_name
            ico_name= row[0]
            # specify parameters as a dictionary
            payload={'f':'tweets',  # retrieve tweets
                     'q':'"{}"'.format(ico_name) + ' ico' + ' since:' + row[1] + ' until:' + row[2], # query string
                     'max_position': min_position} 

            # send a request with parameters and headers
            r = requests.get("https://twitter.com/i/search/timeline",\
                          params=payload, headers=headers)

            # this is equivalent to type the URL
            if r.status_code==200:
                result=r.json()

                # get html source code of tweets
                tweets_html = result['items_html']

                # Search returns tweets in the decending order of time
                # retrieve the position of the earliest tweets returned
                min_position_updated = result['min_position']
                # update tweets list
                tweets_scraped += getTweets(tweets_html)  

        # reset position parameter to keep the loop going        
        min_position = '1'
        min_position_updated = ''
        
        tweets = pd.DataFrame(tweets_scraped)
        
    return tweets

In [8]:
# pre-sale stage
tweets_pre = tweets_scraper(df_pre)
tweets_pre.shape
tweets_pre.head(10)
## save results to csv file
tweets_pre.to_csv('tweets_pre.csv', index = False, encoding = 'utf-8-sig')

# sale stage
tweets_sale = tweets_scraper(df_sale)
tweets_sale.shape
tweets_sale.head(10)
## save results to csv file
tweets_sale.to_csv('tweets_sale.csv', index = False, encoding = 'utf-8-sig')

(244181, 8)

,ICO_name,date,language,like,reply,retweet,text,user
0,XinFin,2017-07-13 04:16:23,en,1,1,,"Hi, the pre sale ICO has already ended. If you...",XinFin_Official
1,XinFin,2017-07-12 01:58:22,en,1,,,Thanks for the wonderful words :) Truly loved ...,XinFin_Official
2,XinFin,2017-07-11 12:57:34,en,,,,Join Xinfin’s #Bitcoin Ecosystem & Bridge the ...,bitcoinewsboard
3,XinFin,2017-07-11 10:09:08,en,2,1,,Thanks for xinfin and dim BUT hive is a scam.....,GlendaleAg
4,XinFin,2017-07-11 08:31:04,en,2,,,thanks for mentioning us :) it's a great achie...,XinFin_Official
5,XinFin,2017-06-15 08:31:21,en,,,,Join Xinfin’s Bitcoin Ecosystem & Bridge the G...,coinspectator
6,Cofound.it,2017-06-06 18:20:00,en,1,,,"Crazy, #cofoundit ICO FOMO moods! 97% already ...",AndyLenz
7,Cofound.it,2017-06-06 17:06:14,en,5,,4,@cofound_it Succes ICO presale 96% collected ...,CyrilGaab
8,Cofound.it,2017-06-06 16:40:10,en,,,,CoFound.it ICO 95% complete 1 day before launc...,Cryptogeeks
9,Cofound.it,2017-06-06 12:21:52,en,,,,“ICO Review of Cofound.it (CFI tokens)” by @Et...,s_chmel


(702085, 8)

,ICO_name,date,language,like,reply,retweet,text,user
0,Waves,2016-05-30 18:07:02,pl,3,,2,#Waves Platform - Post ICO calendar\nhttps://b...,MagicKomodo
1,Waves,2016-05-30 15:34:10,en,1,,,View individual investments for #rise ico here...,CryptoPioneer
2,Waves,2016-05-30 15:23:56,en,,,,#waves ico open for 1 more day! #cryptocurrenc...,CryptoChief1111
3,Waves,2016-05-30 13:43:10,en,1,,2,Post ICO calendar — Waves Platform https://blo...,TheEtherFire
4,Waves,2016-05-30 13:02:03,en,,1,,@wavesplatform I sent 5.8 btc to the ICO and I...,LtCrandog
5,Waves,2016-05-30 11:52:01,en,2,,,Less than 2 days left for @wavesplatform ICO! ...,thefunkybits
6,Waves,2016-05-30 11:25:50,en,,,,Dubai + Deloitte partnerships + Former Masterc...,FedKassad
7,Waves,2016-05-30 10:30:24,en,2,,1,Currently Active #ICO #Crowdsales #waves #rise...,TheEtherFire
8,Waves,2016-05-30 09:15:21,de,,,,Waves ICO ist wegen Bitcoin Anstieg gefährlich...,Altcoinspekulan
9,Waves,2016-05-30 06:28:08,en,,,,@wavesplatform Waves' ICO has collected until ...,BitcoinBjoern


## Analysis
### 1. Data manipulation

In [50]:
# check if empty text exists
tweets_pre.text[tweets_pre.text.isna() == True].shape
#tweets_pre.text[tweets_pre.text.isna() == True].index
tweets_sale.text[tweets_sale.text.isna() == True].shape
#tweets_sale.text[tweets_sale.text.isna() == True].index
# assign new datasets for analyses
df_pre = tweets_pre
df_sale = tweets_sale

(0,)

(0,)

### 2. Text cleaning

In [51]:
# Clean texts, remove hashtags, at signs, and links by regex
def text_cleaner(data):
    raw_text = data.text.tolist()
    text = []
    for item in raw_text:
        cleaned = ' '.join(re.sub("([^A-Za-z0-9!%&-+=:;/',.?])|(\w+:\/\/\S+)|(\w+.\w+.\w+\/\S+)", " ", item).split())
        text.append(cleaned)    
    
    return text

## pre-sale stage
df_pre = df_pre.assign(text = text_cleaner(df_pre))
df_pre.text.head().values.tolist()

## sale stage
df_sale = df_sale.assign(text = text_cleaner(df_sale))
df_sale.text.head().values.tolist()

['Hi, the pre sale ICO has already ended. If you wish to purchase XDC ICO, you can get in touch with us at info xinfin.org XinfinF',
 'Thanks for the wonderful words :) Truly loved it :)',
 'Join Xinfin s Bitcoin Ecosystem & Bridge the Global Infrastructure Deficit ico sponsored xinfin',
 'Thanks for xinfin and dim BUT hive is a scam...they just copied PPT white paper word for word',
 "thanks for mentioning us :) it's a great achievement for us :)"]

['Waves Platform Post ICO calendar wavesplatform Blockchain Token News tech',
 'View individual investments for rise ico here.. 444 btc raised so far, 10% bonus is almost over lisk waves eth',
 'waves ico open for 1 more day! cryptocurrency blockchain Ethereum Bitcoin',
 'Post ICO calendar Waves Platform',
 'wavesplatform I sent 5.8 btc to the ICO and I only have 26837 waves. That seems less than I should have gotten. Is something up?']

### 3. Sentiment analysis

In [54]:
# Use Vader method to get sentiment_score
# it's a compound score, the sum of all of the lexicon ratings which have been standardized to range between -1 and 1
def sentiment_scorer(data):
    sid = SentimentIntensityAnalyzer()
    sentiment_score = []
    for tweet in data.text:
        ss = sid.polarity_scores(tweet)
        sentiment_score.append(ss['compound'])
    
    return sentiment_score
   
## pre-sale stage
df_pre = df_pre.assign(sentiment_score = sentiment_scorer(df_pre))
df_pre.head()

## sale stage
df_sale = df_sale.assign(sentiment_score = sentiment_scorer(df_sale))
df_sale.head()

,ICO_name,date,language,like,reply,retweet,text,user,sentiment_score
0,XinFin,2017-07-13 04:16:23,en,1,1,,"Hi, the pre sale ICO has already ended. If you...",XinFin_Official,0.4019
1,XinFin,2017-07-12 01:58:22,en,1,,,Thanks for the wonderful words :) Truly loved ...,XinFin_Official,0.9607
2,XinFin,2017-07-11 12:57:34,en,,,,Join Xinfin s Bitcoin Ecosystem & Bridge the G...,bitcoinewsboard,-0.1280
3,XinFin,2017-07-11 10:09:08,en,2,1,,Thanks for xinfin and dim BUT hive is a scam.....,GlendaleAg,0.2382
4,XinFin,2017-07-11 08:31:04,en,2,,,thanks for mentioning us :) it's a great achie...,XinFin_Official,0.9186


,ICO_name,date,language,like,reply,retweet,text,user,sentiment_score
0,Waves,2016-05-30 18:07:02,pl,3,,2,Waves Platform Post ICO calendar wavesplatform...,MagicKomodo,0.0000
1,Waves,2016-05-30 15:34:10,en,1,,,View individual investments for rise ico here....,CryptoPioneer,0.5809
2,Waves,2016-05-30 15:23:56,en,,,,waves ico open for 1 more day! cryptocurrency ...,CryptoChief1111,0.0000
3,Waves,2016-05-30 13:43:10,en,1,,2,Post ICO calendar Waves Platform,TheEtherFire,0.0000
4,Waves,2016-05-30 13:02:03,en,,1,,wavesplatform I sent 5.8 btc to the ICO and I ...,LtCrandog,0.0000


### 4. Data Aggregation

In [118]:
# group data by ICO names
pre_g = df_pre.groupby('ICO_name')
sale_g = df_sale.groupby('ICO_name')
print('{} ICOs included during respective pre-sale periods'.format(pre_g.first().shape[0]))
print('{} ICOs included during respective sale periods'.format(sale_g.first().shape[0]))

# convert the following columns to numeric values
df_pre.like = pd.to_numeric(df_pre.like)
df_pre.reply = pd.to_numeric(df_pre.reply)
df_pre.retweet = pd.to_numeric(df_pre.retweet)
df_sale.like = pd.to_numeric(df_sale.like)
df_sale.reply = pd.to_numeric(df_sale.reply)
df_sale.retweet = pd.to_numeric(df_sale.retweet)

df_pre.head(10)
df_sale.head(10)

708 ICOs included during respective pre-sale periods
1519 ICOs included during respective sale periods


,ICO_name,date,language,like,reply,retweet,text,user,sentiment_score
0,XinFin,2017-07-13 04:16:23,en,1.0,1.0,NaN,"Hi, the pre sale ICO has already ended. If you...",XinFin_Official,0.4019
1,XinFin,2017-07-12 01:58:22,en,1.0,NaN,NaN,Thanks for the wonderful words :) Truly loved ...,XinFin_Official,0.9607
2,XinFin,2017-07-11 12:57:34,en,NaN,NaN,NaN,Join Xinfin s Bitcoin Ecosystem & Bridge the G...,bitcoinewsboard,-0.1280
3,XinFin,2017-07-11 10:09:08,en,2.0,1.0,NaN,Thanks for xinfin and dim BUT hive is a scam.....,GlendaleAg,0.2382
4,XinFin,2017-07-11 08:31:04,en,2.0,NaN,NaN,thanks for mentioning us :) it's a great achie...,XinFin_Official,0.9186
5,XinFin,2017-06-15 08:31:21,en,NaN,NaN,NaN,Join Xinfin s Bitcoin Ecosystem & Bridge the G...,coinspectator,-0.1280
6,Cofound.it,2017-06-06 18:20:00,en,1.0,NaN,NaN,"Crazy, cofoundit ICO FOMO moods! 97% already s...",AndyLenz,-0.4559
7,Cofound.it,2017-06-06 17:06:14,en,5.0,NaN,4.0,cofound it Succes ICO presale 96% collected IC...,CyrilGaab,0.0000
8,Cofound.it,2017-06-06 16:40:10,en,NaN,NaN,NaN,CoFound.it ICO 95% complete 1 day before launc...,Cryptogeeks,0.0000
9,Cofound.it,2017-06-06 12:21:52,en,NaN,NaN,NaN,ICO Review of Cofound.it (CFI tokens) by Ether...,s_chmel,0.0000


,ICO_name,date,language,like,reply,retweet,text,user,sentiment_score
0,Waves,2016-05-30 18:07:02,pl,3.0,NaN,2.0,Waves Platform Post ICO calendar wavesplatform...,MagicKomodo,0.0000
1,Waves,2016-05-30 15:34:10,en,1.0,NaN,NaN,View individual investments for rise ico here....,CryptoPioneer,0.5809
2,Waves,2016-05-30 15:23:56,en,NaN,NaN,NaN,waves ico open for 1 more day! cryptocurrency ...,CryptoChief1111,0.0000
3,Waves,2016-05-30 13:43:10,en,1.0,NaN,2.0,Post ICO calendar Waves Platform,TheEtherFire,0.0000
4,Waves,2016-05-30 13:02:03,en,NaN,1.0,NaN,wavesplatform I sent 5.8 btc to the ICO and I ...,LtCrandog,0.0000
5,Waves,2016-05-30 11:52:01,en,2.0,NaN,NaN,Less than 2 days left for wavesplatform ICO! W...,thefunkybits,0.0000
6,Waves,2016-05-30 11:25:50,en,NaN,NaN,NaN,Dubai + Deloitte partnerships + Former Masterc...,FedKassad,0.0000
7,Waves,2016-05-30 10:30:24,en,2.0,NaN,1.0,Currently Active ICO Crowdsales waves rise,TheEtherFire,0.4019
8,Waves,2016-05-30 09:15:21,de,NaN,NaN,NaN,Waves ICO ist wegen Bitcoin Anstieg gef hrlich...,Altcoinspekulan,-0.5994
9,Waves,2016-05-30 06:28:08,en,NaN,NaN,NaN,wavesplatform Waves' ICO has collected until t...,BitcoinBjoern,0.0000


#### a) count unique tweets of each ICO

In [75]:
def uniqueText_counter(data):
    grouped = data.groupby('ICO_name')
    # create a list of lists of tweets on ICO level
    text_total = [] 
    for key, item in grouped:
        text = []
        text_total.append(text)
        for item in grouped.get_group(key)['text'].values:
            text.append(item)
    # compare pairwised Levenshtein Distance
    # considering a tweet as unique if the distance does not exceed the threshold
    # count the length of unique tweets
    unique_tweets = []
    for texts in text_total:
        unique = []
        sort = sorted(texts)
        while len(sort) > 1:
            s1 = sort[0]
            s2 = sort[1]
            if fuzz.token_set_ratio(s1, s2) >= 70:
                sort.remove(sort[1])
            else:
                unique.append(sort[0])
                sort.remove(sort[0])
        else:
            unique.append(sort[0])
        unique_tweets.append(len(unique))
    
    return unique_tweets

pre_unit = uniqueText_counter(df_pre)
sale_unit = uniqueText_counter(df_sale)

#### b) aggregate other features

In [294]:
def ico_agg(data):
    df = pd.DataFrame()
    grouped = data.groupby('ICO_name')
    # get aggregated data on ICO level 
    g_like = grouped.like.sum().reset_index() # total likes
    g_reply = grouped.reply.sum().reset_index().iloc[:,1] # total replies    
    g_retweet = grouped.retweet.sum().reset_index().iloc[:,1] # total retweets
    g_user = grouped.user.count().reset_index().iloc[:,1] # unique users
    senti_avg = grouped.sentiment_score.mean().reset_index().iloc[:,1].rename('senti_avg') # average sentiment score
    senti_std = grouped.sentiment_score.agg(np.std).reset_index().iloc[:,1].rename('senti_std') # standard deviation of sentiment score
    lang = grouped.language.value_counts().groupby('ICO_name').count().reset_index().iloc[:,1] # amount of language used
    tweets = uniqueText_counter(data) # unique tweets
    # put all results into a new dataframe
    df = pd.concat([df, g_like], axis = 1)    
    for item in [g_reply, g_retweet, g_user, senti_avg, senti_std, lang, tweets]:
        df = pd.concat([df, item], axis = 1)

    return df

#### c) save to csv file

In [297]:
pre_agg = ico_agg(df_pre)
pre_agg.shape
pre_agg.head(10)
sale_agg = ico_agg(df_sale)
sale_agg.shape
sale_agg.head(10)

# save to csv files
pre_agg.to_csv('pre_agg.csv', index = False, encoding = 'utf-8-sig')
sale_agg.to_csv('sale_agg.csv', index = False, encoding = 'utf-8-sig')

(708, 9)

,ICO_name,like,reply,retweet,user,senti_avg,senti_std,language,tweets
0,0xcert,31.0,0.0,15.0,13,0.235754,0.371734,4,9
1,3D-Token,6.0,0.0,2.0,3,0.584467,0.090817,1,2
2,4NEW,1706.0,38.0,1551.0,437,0.151787,0.389988,8,186
3,ABLE,1064.0,81.0,453.0,914,0.470913,0.374425,6,631
4,AEN,1531.0,14.0,1662.0,79,0.236891,0.346561,8,49
5,AICrypto,8.0,0.0,2.0,2,0.229400,0.324421,2,2
6,AIDA,62.0,22.0,15.0,326,0.385780,0.374140,7,216
7,ARAW,8423.0,165.0,2411.0,4646,0.183872,0.332302,18,717
8,ARROUND,229.0,3.0,205.0,144,0.072753,0.371342,6,42
9,AXenS,464.0,57.0,380.0,74,0.266750,0.313796,4,56


(1519, 9)

,ICO_name,like,reply,retweet,user,senti_avg,senti_std,language,tweets
0,0x,493.0,72.0,281.0,597,0.046177,0.195695,12,213
1,0xcert,30.0,7.0,6.0,25,0.050952,0.224697,6,17
2,1World,48.0,1.0,18.0,41,0.141232,0.268671,4,29
3,2100NEWSICO,0.0,0.0,0.0,2,0.409600,0.579262,1,2
4,300Token,21.0,6.0,21.0,6,0.280817,0.257848,1,4
5,300cubitsTEU,0.0,0.0,0.0,1,0.000000,NaN,1,1
6,3D-Token,198.0,100.0,82.0,739,0.029256,0.104052,5,125
7,3cCoin,4.0,1.0,3.0,5,0.145380,0.325080,2,5
8,4NEW,621.0,10.0,587.0,208,0.294413,0.402930,10,120
9,7ype,0.0,0.0,0.0,5,0.453460,0.418110,1,4
